In [1]:
import pandas as pd
import os
from difflib import get_close_matches


In [2]:
dfs_clf = [pd.read_csv(os.path.join('data', df_clf)).rename(columns={'JOB': 'job', 'answer': 'job_clf'}) for df_clf in os.listdir('data') if df_clf.endswith('_clf.csv')]

In [3]:
dfs_clf = [pd.read_csv(os.path.join('data', df_clf)).rename(columns={'JOB': 'job', 'answer': 'job_clf'}).dropna() for df_clf in os.listdir('data') if df_clf.endswith('_clf.csv')]
df_salaire = pd.read_excel('data/SHS_TM.xlsx',index_col=0, header=1)

In [4]:
"On doit faire ça psq le LLM classifie correctement, mais par exemple vêtem, lingerie, chauss, literie va etre classifier comme vêtement, lingerie, chaussure, litterie"
def find_closest_job(job):
    matches = get_close_matches(job, df_salaire.columns, n=1, cutoff=0.5)
    if matches:
        col = matches[0]
    else:
        col = None
    return col

In [5]:
for year, df in zip([1890, 1901, 1923, 1951], dfs_clf):
    df['job_clf_real'] = df.job_clf.apply(lambda x: find_closest_job(x))
    df['salaire'] = df.job_clf_real.apply(lambda x: df_salaire[x].loc[year])
    df = df.drop(columns=["Unnamed: 0", "job_clf"]).rename(columns={'job_clf_real':'job_clf'})
    df.to_csv(f'data/table_{year}_clf.csv')